In [ ]:
# !pip3 install qwen-vl-utils torch torchvision jupyter gradio accelerate

In [ ]:
import torch

from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "CPU only")
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
Current device: 0
Device name: NVIDIA L40S


##### Load the Model

In [2]:
MODEL_PATH = "/common/public/Qwen2.5-VL/Qwen2.5-VL-7B-Instruct"

In [3]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_PATH, 
    dtype=torch.bfloat16, 
    # device_map="auto"
).to("cuda" if torch.cuda.is_available() else "cpu")

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     MODEL_PATH,
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

##### Prepare Inputs for Inference

- The default range for the number of visual tokens per image in the model is 4-16384.
- You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
- min_pixels = 256*28*28
- max_pixels = 1280*28*28
- processor = AutoProcessor.from_pretrained(MODEL_PATH, min_pixels=min_pixels, max_pixels=max_pixels)

In [4]:
# default processer
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(MODEL_PATH)

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


##### Prepare Inputs for Inference as LLM Message

In [5]:
INPUT_PROMPT = "Describe the start of the video"
IMAGE_PATH = r"./src/test_files/images.jpeg"
VIDEO_PATH = r"./src/data/video_clip/0102_scene_000_central_perk.mp4"

In [27]:
### IMAGES
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": IMAGE_PATH,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
prompt_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages, )
# Pass inputs to processor
inputs = processor(
    text=[prompt_text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
### VIDEO
FPS = 8.0

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": VIDEO_PATH,
                "max_pixels": 360 * 420,
                "fps": FPS,
            },
            {
                "type": "text", 
                "text": "Select the correct answer to what is happening in this video. A) The video shows two baboons in a zoo enclosure\nB) There are two birds in a cage"
            },
        ],
    }
]

# Preparation for inference
prompt_text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
print(video_kwargs)

inputs = processor(
    text=[prompt_text],
    images=image_inputs,
    videos=video_inputs,
    # fps=FPS,
    padding=True,
    return_tensors="pt",
    **video_kwargs,
)
inputs = inputs.to("cuda")

qwen-vl-utils using decord to read video.
smart_nframes: nframes[544.0] > total_frames[204]


{'do_sample_frames': False, 'fps': [3.0]}


##### Run Inference

https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct

In [7]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The video does not show any baboons or birds; instead, it depicts a scene from the TV show "Friends" where characters are sitting and interacting in a café setting. Therefore, neither option A nor B accurately describes the content of the video.']


In [ ]:
gc.collect()
torch.cuda.empty_cache()